In [1]:
import pandas as pd

#### Load all sheets in the underlying file

In [2]:
file_path = 'Novel Coronavirus (2019-nCoV) Cases, provided by JHU CSSE.xlsx'

In [3]:
main_df = pd.concat(pd.read_excel(file_path, sheet_name= None), ignore_index = True)

/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [4]:
# Merge duplicate columns with different names

main_df.loc[main_df.Deaths.isna(),'Deaths'] =  main_df.loc[main_df.Deaths.isna(),'Demised']

main_df.loc[main_df['Country/Region'].isna(),'Country/Region'] =  main_df.loc[main_df['Country/Region'].isna(),'Country']

main_df.loc[main_df['Last Update'].isna(),'Last Update'] = main_df.loc[main_df['Last Update'].isna(),'Date last updated']

main_df = main_df.loc[:, ~main_df.columns.isin(['Demised' , 'Country' , 'Date last updated'])]
main_df

,Confirmed,Country/Region,Deaths,Last Update,Province/State,Recovered,Suspected
0,7153.0,Mainland China,249.0,2020-01-31 19:00:00,Hubei,169.0,NaN
1,537.0,Mainland China,NaN,2020-01-31 19:00:00,Zhejiang,14.0,NaN
2,436.0,Mainland China,NaN,2020-01-31 19:00:00,Guangdong,10.0,NaN
3,352.0,Mainland China,2.0,2020-01-31 19:00:00,Henan,3.0,NaN
4,332.0,Mainland China,NaN,2020-01-31 19:00:00,Hunan,3.0,NaN
...,...,...,...,...,...,...,...
1120,NaN,China,NaN,2020-01-21 00:00:00,Heilongjiang,NaN,1.0
1121,1.0,Japan,NaN,2020-01-21 00:00:00,NaN,NaN,NaN
1122,2.0,Thailand,NaN,2020-01-21 00:00:00,NaN,NaN,NaN
1123,1.0,South Korea,NaN,2020-01-21 00:00:00,NaN,NaN,NaN


In [5]:
# merge duplicate country names
main_df['Country/Region'] = main_df['Country/Region'].str.replace(r'^China','Mainland China')
main_df['Country/Region'] = main_df['Country/Region'].str.replace(r'United States','US')

main_df['day'] = main_df['Last Update'].dt.day
main_df = main_df.fillna(0) # will take max value in the next step

In [6]:
clean_df =main_df.groupby(['Country/Region','Province/State','day']).agg({'Last Update':'max',
                                                                  'Confirmed':'max',
                                                                  'Suspected':'max',
                                                                  'Recovered':'max',
                                                                  'Deaths':'max'}).reset_index()

In [7]:
clean_df[clean_df['Province/State'] =='Hubei']

,Country/Region,Province/State,day,Last Update,Confirmed,Suspected,Recovered,Deaths
215,Mainland China,Hubei,21,2020-01-21 00:00:00,270.0,11.0,0.0,0.0
216,Mainland China,Hubei,22,2020-01-22 12:00:00,444.0,0.0,0.0,0.0
217,Mainland China,Hubei,23,2020-01-23 12:00:00,444.0,0.0,28.0,17.0
218,Mainland China,Hubei,24,2020-01-24 12:00:00,549.0,0.0,31.0,24.0
219,Mainland China,Hubei,25,2020-01-25 22:00:00,1052.0,0.0,42.0,52.0
220,Mainland China,Hubei,26,2020-01-26 23:00:00,1423.0,0.0,44.0,76.0
221,Mainland China,Hubei,27,2020-01-27 20:30:00,2714.0,0.0,47.0,100.0
222,Mainland China,Hubei,28,2020-01-28 23:00:00,3554.0,0.0,80.0,125.0
223,Mainland China,Hubei,29,2020-01-29 21:00:00,4586.0,0.0,90.0,162.0
224,Mainland China,Hubei,30,2020-01-30 21:30:00,5806.0,0.0,116.0,204.0


In [8]:
clean_df['Last Update'].value_counts().sort_index()

2020-01-21 00:00:00    27
2020-01-22 12:00:00    38
2020-01-23 12:00:00    46
2020-01-24 12:00:00    40
2020-01-24 16:00:00     1
2020-01-25 12:00:00    42
2020-01-25 22:00:00     2
2020-01-26 23:00:00    47
2020-01-27 09:00:00     1
2020-01-27 20:30:00    52
2020-01-28 18:00:00     1
2020-01-28 23:00:00    51
2020-01-29 21:00:00    56
2020-01-30 21:30:00    59
2020-01-31 19:00:00    63
Name: Last Update, dtype: int64

In [9]:
clean_df[clean_df['Last Update'] == '2020-01-31 19:00:00']

,Country/Region,Province/State,day,Last Update,Confirmed,Suspected,Recovered,Deaths
8,Australia,New South Wales,31,2020-01-31 19:00:00,4.0,0.0,2.0,0.0
11,Australia,Queensland,31,2020-01-31 19:00:00,3.0,0.0,0.0,0.0
16,Australia,Victoria,31,2020-01-31 19:00:00,3.0,0.0,0.0,0.0
22,Cambodia,0,31,2020-01-31 19:00:00,1.0,0.0,0.0,0.0
26,Canada,British Columbia,31,2020-01-31 19:00:00,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
494,US,California,31,2020-01-31 19:00:00,3.0,0.0,0.0,0.0
502,US,Illinois,31,2020-01-31 19:00:00,2.0,0.0,0.0,0.0
513,US,Washington,31,2020-01-31 19:00:00,1.0,0.0,0.0,0.0
516,United Arab Emirates,0,31,2020-01-31 19:00:00,4.0,0.0,0.0,0.0


In [10]:
main_df.loc[main_df['Last Update'] == '2020-01-31 19:00:00','Country/Region'].value_counts()

Mainland China          31
US                       4
Australia                3
Canada                   2
Spain                    1
Vietnam                  1
Singapore                1
UK                       1
Italy                    1
Sri Lanka                1
Finland                  1
Hong Kong                1
Russia                   1
Taiwan                   1
Philippines              1
Japan                    1
India                    1
South Korea              1
Thailand                 1
Cambodia                 1
Macau                    1
Nepal                    1
Sweden                   1
France                   1
Malaysia                 1
United Arab Emirates     1
Germany                  1
Name: Country/Region, dtype: int64

In [11]:
main_df[main_df['Last Update'] == '2020-01-31 19:00:00']

,Confirmed,Country/Region,Deaths,Last Update,Province/State,Recovered,Suspected,day
0,7153.0,Mainland China,249.0,2020-01-31 19:00:00,Hubei,169.0,0.0,31
1,537.0,Mainland China,0.0,2020-01-31 19:00:00,Zhejiang,14.0,0.0,31
2,436.0,Mainland China,0.0,2020-01-31 19:00:00,Guangdong,10.0,0.0,31
3,352.0,Mainland China,2.0,2020-01-31 19:00:00,Henan,3.0,0.0,31
4,332.0,Mainland China,0.0,2020-01-31 19:00:00,Hunan,3.0,0.0,31
...,...,...,...,...,...,...,...,...
58,2.0,Italy,0.0,2020-01-31 19:00:00,0,0.0,0.0,31
59,1.0,Sweden,0.0,2020-01-31 19:00:00,0,0.0,0.0,31
60,2.0,Russia,0.0,2020-01-31 19:00:00,0,0.0,0.0,31
61,1.0,Spain,0.0,2020-01-31 19:00:00,0,0.0,0.0,31


#### Alternative method choose worksheet before merging

In [12]:
xl = pd.ExcelFile(file_path)
xl.sheet_names

['Jan31_7pm',
 'Jan31_2pm',
 'Jan30_930pm',
 'Jan30_11am',
 'Jan29_9pm',
 'Jan29_230pm',
 'Jan29_130pm',
 'Jan28_11pm',
 'Jan28_6pm',
 'Jan28_1pm',
 'Jan27_830pm',
 'Jan27_7pm',
 'Jan27_9am',
 'Jan26_11pm',
 'Jan26_11am',
 'Jan25_10pm',
 'Jan25_12pm',
 'Jan25_12am',
 'Jan24_12pm',
 'Jan24_12am',
 'Jan23_12pm',
 'Jan22_12pm',
 'Jan22_12am']

In [13]:
# use the latest time for each date
split_name_df = pd.DataFrame(data = list(map(lambda x: x.split('_'), xl.sheet_names)), columns =['date','hour'])
split_name_df

,date,hour
0,Jan31,7pm
1,Jan31,2pm
2,Jan30,930pm
3,Jan30,11am
4,Jan29,9pm
5,Jan29,230pm
6,Jan29,130pm
7,Jan28,11pm
8,Jan28,6pm
9,Jan28,1pm


In [14]:
# extract time components
split_name_df['suffix'] = split_name_df.hour.apply(lambda x: x[-2:])

split_name_df['hour_digit'] = split_name_df.hour.str.extract(r'(?P<hour>[\d]+)')

split_name_df['date_digit'] = split_name_df.date.str.extract('([\d]+)').astype(int)

In [15]:
# deal with special case of 12am and half-hour

split_name_df.loc[split_name_df.hour == '12am', 'date_digit'] = split_name_df.loc[split_name_df.hour == '12am', 'date_digit'] - 1

split_name_df['hour_digit'] = split_name_df.hour_digit.str.replace('30','').astype(int)

split_name_df.loc[split_name_df.hour == '12am', 'hour_digit'] = 24

split_name_df.loc[split_name_df.hour == '12am', 'suffix'] = 'pm'

In [16]:
# filter for the last file for each day

filter_name_df = split_name_df[split_name_df.suffix == 'pm']

filter_name_df = filter_name_df.groupby('date_digit').agg({'hour_digit':'max','suffix': 'max'}).reset_index()

filter_name_df

,date_digit,hour_digit,suffix
0,21,24,pm
1,22,12,pm
2,23,24,pm
3,24,24,pm
4,25,12,pm
5,26,11,pm
6,27,8,pm
7,28,11,pm
8,29,9,pm
9,30,9,pm


In [17]:
filter_name_df = pd.merge(split_name_df,filter_name_df, how = 'right', 
         left_on = ['date_digit', 'hour_digit','suffix'], right_on = ['date_digit','hour_digit','suffix'])

filter_name = (filter_name_df.date + '_' + filter_name_df.hour).to_list()
filter_name

['Jan31_7pm',
 'Jan30_930pm',
 'Jan29_9pm',
 'Jan28_11pm',
 'Jan27_830pm',
 'Jan26_11pm',
 'Jan25_12pm',
 'Jan25_12am',
 'Jan24_12am',
 'Jan22_12pm',
 'Jan22_12am']

In [18]:
main_2_df = pd.concat(pd.read_excel(file_path, sheet_name= filter_name), ignore_index = True)

/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [19]:
main_2_df['Last Update'].value_counts()

2020-01-31 19:00:00    63
2020-01-30 21:30:00    59
2020-01-29 21:00:00    56
2020-01-27 20:30:00    52
2020-01-28 23:00:00    51
2020-01-26 23:00:00    47
2020-01-25 12:00:00    44
2020-01-25 00:00:00    44
2020-01-22 12:00:00    38
2020-01-24 00:00:00    38
2020-01-28 18:00:00     1
Name: Last Update, dtype: int64

In [20]:
# def read_excel_sheets(xls_path):
#     """Read all sheets of an Excel workbook and return a single DataFrame"""
#     print(f'Loading {xls_path} into pandas')
#     xl = pd.ExcelFile(xls_path)
#     df = pd.DataFrame()
#     columns = None
#     for idx, name in enumerate(xl.sheet_names):
#         print(f'Reading sheet #{idx}: {name}')
#         sheet = xl.parse(name)
#         if idx == 0:
#             # Save column names from the first sheet to match for append
#             columns = sheet.columns
#         sheet.columns = columns
#         # Assume index of existing data frame when appended
#         df = df.append(sheet, ignore_index=True)
#     return df

# read_excel_sheets(file_name)